## Mirandata - evasão em cursos de engenharia

In [28]:
import pandas as pd
import numpy as np
%matplotlib inline

par_datas = ((2009,2013), (2010, 2014), (2011,2015), (2012, 2016))


def evasao_total(df_filtrado, par_datas):
    ev = evasao_ponderada(df_filtrado, par_datas)
    total = sum(ev)
    print(total)
    return total

def evasao_ponderada(df_filtrado, par_datas):
    #print(df_filtrado.NO_CURSO.value_counts())
    df = df_filtrado
    ev = calcula_evasao(df, par_datas)
    d = pd.DataFrame(ev)
    d_pos = d[d[2]>0] # Excluí os casos em que o numero de vagas diminuiu. São ~ 0.5% das IEs
    # Evasao = peso da IE * taxa de evasao
    d_pos_evasao = d_pos[2]*d_pos[3]
    d_pos_evasao= d_pos_evasao/np.sum(d_pos[3])
    return d_pos_evasao
    
    
def calcula_evasao(df_filtrado, par_datas):
    df = df_filtrado
    gb = df.groupby(["CO_IES",  "CO_CURSO", "ano_censo"]).sum()
    gb_mini = df.groupby(["CO_IES",  "CO_CURSO"]).sum()
    evasao = []
    for par in gb_mini.index:
        for datas in par_datas:
            evasao_par = calcula_evasao_gb(gb, par[0], par[1], datas[0], datas[1])
            if evasao_par != None:
                evasao.append(evasao_par)   
    return evasao
    
       
def calcula_evasao_gb(gb, ies, curso, ano_inicial, ano_final):
    #print(gb.loc[(ies, curso),:].index ) 
    if (ano_inicial in gb.loc[(ies, curso),:].index ) and (ano_final in gb.loc[(ies, curso),:].index):
        if pd.notnull(gb.loc[(ies,curso,ano_final),:].QT_CONCLUINTE_CURSO) and pd.notnull(gb.loc[(ies,curso,ano_inicial),:].QT_INGRESSO_CURSO):
            saida = gb.loc[(ies,curso,ano_final),:].QT_CONCLUINTE_CURSO
            entrada = gb.loc[(ies,curso,ano_inicial),:].QT_INGRESSO_CURSO
            evasao = (entrada - saida)/entrada
            evasao_peso = entrada
            #print("saida: {0}, ingresso{1} ".format(saida, entrada))
            if evasao_peso!=0:
                #Não teve ingresso 5 anos atrás
                return ies, curso, evasao, evasao_peso
            else:
                return None
        else:
            #print("Não foi possível calcular evasão")
            pass

    return None
    



In [29]:
dados = pd.read_excel("engenharias_2009_a_2016.xlsx")

In [30]:
comps = dados.loc[dados.NO_CURSO.str.contains("COMPUTAÇÃO"),:]
mec = dados.loc[dados.NO_CURSO.str.contains("MECÂNICA"),:]
mex = dados.loc[dados.NO_CURSO.str.contains("TRÔNICA")|dados.NO_CURSO.str.contains("CONTROLE") ,:]

In [31]:
comps.NO_CURSO.value_counts()

ENGENHARIA DE COMPUTAÇÃO                                   862
ENGENHARIA DA COMPUTAÇÃO                                   261
ENGENHARIA DE COMPUTAÇÃO E INFORMAÇÃO                        7
ENGENHARIA DE COMPUTAÇÃO - ÊNFASE SISTEMAS CORPORATIVOS      3
ENGENHARIA DE COMPUTAÇÃO - ENGENHARIA DA COMPUTAÇÃO          3
ENGENHARIA DA COMPUTAÇÃO - COMPUTAÇÃO APLICADA               2
ENGENHARIA DA COMPUTAÇÃO - AUTOMAÇÃO INDUSTRIAL              2
ENGENHARIA DA COMPUTAÇÃO - SISTEMAS E COMPUTAÇÃO             2
ENGENHARIA DA COMPUTAÇÃO - TELEMÁTICA                        1
Name: NO_CURSO, dtype: int64

In [32]:
mec.NO_CURSO.value_counts()

ENGENHARIA MECÂNICA                                    2080
ENGENHARIA MECÂNICA AERONÁUTICA                          13
ENGENHARIA MECÂNICA - ÊNFASE EM MATERIAIS METÁLICOS       7
ENGENHARIA MECÂNICA EMPRESARIAL                           7
ENGENHARIA MECÂNICA - ÊNFASE EM MECÂNICA PLENA            7
ENGENHARIA MECÂNICA E DE AUTOMÓVEIS                       7
ENGENHARIA MECÂNICA - ÊNFASE EM PROJETOS                  6
ENGENHARIA MECÂNICA - ENGENHARIA MECÂNICA                 6
ENGENHARIA MECÂNICA E DE ARMAMENTO                        5
ENGENHARIA MECÂNICA - ÊNFASE EM AERONAVES                 4
ABI - ENGENHARIA MECÂNICA                                 3
ENGENHARIA MECÂNICA DE ARMAMENTO                          2
ENGENHARIA INDUSTRIAL MECÂNICA                            2
ENGENHARIA MECÂNICA - MECÂNICA E AUTOMOBILÍSTICA          2
ENGENHARIA MECÂNICA - CONTROLE E AUTOMAÇÃO                2
ENGENHARIA MECÂNICA - ÊNFASE EM MECATRÔNICA               2
ENGENHARIA - ENGENHARIA MECÂNICA        

In [33]:
mex.NO_CURSO.value_counts()

ENGENHARIA DE CONTROLE E AUTOMAÇÃO                                         966
ENGENHARIA MECATRÔNICA                                                     187
ENGENHARIA DE CONTROLE E AUTOMACAO                                          41
ENGENHARIA DE CONTROLE E AUTOMAÇÃO - MECATRÔNICA                            38
ENGENHARIA - ENGENHARIA DE CONTROLE E AUTOMACAO                             11
ENGENHARIA DE AUTOMAÇÃO E CONTROLE                                          11
ENGENHARIA DE CONTROLE E AUTOMACAO  MECATRONICA                             10
ENGENHARIA DE CONTROLE E AUTOMACAO MECATRONICA                               8
ENGENHARIA DE MECATRÔNICA                                                    7
ENGENHARIA DE CONTROLE E AUTOMAÇÃO DE PROCESSOS                              7
MECATRÔNICA                                                                  3
ENGENHARIA - ENGENHARIA DE AUTOMACAO E CONTROLE                              2
ENGENHARIA MECATRÔNICA - ENGENHARIA DE CONTROLE E AU

### Escolas em geral no Brasil para os 3 cursos

In [34]:
ev_comp = evasao_total(comps, par_datas)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.753604397884


In [35]:
ev_mec = evasao_total(mec, par_datas)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.685985302549


In [36]:
ev_mex = evasao_total(mex, par_datas)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


0.726458390932


In [13]:
sum(ev_mex)

0.7264583909317115

### Escolas no estado de São Paulo

In [42]:
comps_sp = comps.loc[comps.CO_UF_CURSO==35,:]
ev_comps_sp = evasao_total(comps_sp, par_datas)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.671163575042


In [43]:
mec_sp = mec.loc[mec.CO_UF_CURSO==35,:]
ev_mec_sp = evasao_total(mec_sp, par_datas)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.679704825266


In [44]:
mex_sp =  mex.loc[mex.CO_UF_CURSO==35,:]
ev_mex_sp = evasao_total(mex_sp, par_datas)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


0.704744983278


### Universidades públicas em geral

Computação

In [49]:
comps_pub = comps.loc[(comps.CO_CATEGORIA_ADMINISTRATIVA==1) | (comps.CO_CATEGORIA_ADMINISTRATIVA==2) | (comps.CO_CATEGORIA_ADMINISTRATIVA==3),:]
ev_comps_pub = evasao_total(comps_pub, par_datas)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.721627099016


Mecânicas

In [51]:
mecs = mec
mecs_pub = mecs.loc[(mecs.CO_CATEGORIA_ADMINISTRATIVA==1) | (mecs.CO_CATEGORIA_ADMINISTRATIVA==2) | (mecs.CO_CATEGORIA_ADMINISTRATIVA==3),:]
ev_mecs_pub = evasao_total(mecs_pub, par_datas)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.566971989911


Mecatrônicas

In [52]:

mex_pub = mex.loc[(mex.CO_CATEGORIA_ADMINISTRATIVA==1) | (mex.CO_CATEGORIA_ADMINISTRATIVA==2) | (mex.CO_CATEGORIA_ADMINISTRATIVA==3),:]
ev_mex_pub = evasao_total(mex_pub, par_datas)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.631933717175


### Universidades públicas de SP


In [54]:
def pubsp(dados):
    dados_pubsp = dados.loc[((dados.CO_CATEGORIA_ADMINISTRATIVA==1) | (dados.CO_CATEGORIA_ADMINISTRATIVA==2) | (dados.CO_CATEGORIA_ADMINISTRATIVA==3)) & (dados.CO_UF_CURSO==35),:]
    return evasao_total(dados_pubsp, par_datas)

pubsp(comps)
pubsp(mec)
pubsp(mex)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.433666191155
0.435688866294
0.403846153846


0.4038461538461538

### Universidades privadas de SP

In [55]:
def privsp(dados):
    dados_pubsp = dados.loc[((dados.CO_CATEGORIA_ADMINISTRATIVA==4) | (dados.CO_CATEGORIA_ADMINISTRATIVA==5) ) & (dados.CO_UF_CURSO==35),:]
    return evasao_total(dados_pubsp, par_datas)

privsp(comps)
privsp(mec)
privsp(mex)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.712593942154
0.707236701331


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


0.72483674848


0.72483674848007174

### Estaduais de SP


In [58]:
def estsp(dados):
    dados_pubsp = dados.loc[(dados.CO_CATEGORIA_ADMINISTRATIVA==2)&(dados.CO_UF_CURSO==35),:]
    return evasao_total(dados_pubsp, par_datas)

estsp(comps)
estsp(mec)
estsp(mex)

0.4522417154
0.329888712242
0.293785310734


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in double_scalars


0.29378531073446329